## Campo eléctrico de una distribución de cargas puntuales

In [ ]:
# Hice uso de algunas funciones o argumentos no vistos en clase
# pero todos los lei del manual de numpy y de matplot, tengo claro 
# cada cosa que use a excepcion de np.mgrid en la creacion de la malla,
# no comprendo del todo porque se requiere del uso de numeros complejos
# en los manuales se habla de que esto indica si es inclusivo con el valor de parada
# y en teoria podria presindir de usarlos pero al usar por ejemplo 100 en lugar de 100j 
# el grafico se queda en blanco. 
# Deje por ahi un error respecto a la barra de color, me gustaria corregirlo
# pero ya no me alcanza el tiempo, espero despues poder hacerlo o encontrar si alguien mas lo hizo 
import numpy as np
import matplotlib.pyplot as plt

# Función para calcular el campo eléctrico en cada punto debido a varias cargas
def campo_electrico(cargas, punto):
    # Constante obtenida de internet
    k = 8.9875e9  
    # Crear un vector en zeros, este va a acumular los resultados
    # de los datos que pasen por el ciclo for
    Er = np.zeros(2)  
    
    # Calcular el campo eléctrico debido a cada carga
    for i in range(num_cargas):
        Rpa = cargas[i, 0:2]  # Vector posición de la carga i
        Rpt = punto - Rpa  # Vector posición desde la carga hasta el punto en la malla
        Er += (k * cargas[i, 2] / (np.linalg.norm(Rpt) ** 3)) * Rpt  # Contribucion de la carga i
    
    return Er  # Devolver el campo eléctrico total en el punto

# Pedir al usuario el número de cargas
num_cargas = int(input("Ingrese el número de cargas: "))
# Crear un array vacio para posteriormente llenarlo con los datos de cada carga
xcargas = np.array([])

# Pedir al usuario la información de cada carga
for i in range(num_cargas):
    q = float(input(f"Ingrese el valor de la carga {i + 1} en coulombs: "))
    x = float(input(f"Ingrese la posición x de la carga {i + 1}: "))
    y = float(input(f"Ingrese la posición y de la carga {i + 1}: "))
    ncargas = np.array([x, y, q])
    xcargas = np.append(xcargas, ncargas)
# Crear una matriz con la información de las cargas
cargas = xcargas.reshape((num_cargas, 3))  

# Función para graficar el campo eléctrico
def grafico_campo(a, b, c, d):
    Y, X = np.mgrid[c:d:100j, a:b:100j]  # Crear una malla bidimensional
    # Crear matrices para componentes del campo eléctrico, el uso de zeros_like
    # me permite dar como argumento las matrices que acabo de crear, de este modo
    # me aseguro que la funcion campo electrico pase por cada punto de la malla
    # bidimencional
    Ex, Ey = np.zeros_like(X), np.zeros_like(Y)  

    # Calcular el campo eléctrico en cada punto de la malla, el argumento en shape 
    # da la indicacion de solo mostrar filas o columnas
    for i in range(Y.shape[0]):
        for j in range(X.shape[1]):
            punto = np.array([X[i, j], Y[i, j]])
            campo_en_puntos = campo_electrico(cargas, punto)
            Ex[i, j], Ey[i, j] = campo_en_puntos
    
    # Etiquetar cada carga en el gráfico, uso la funcion zorder
    # para indicar en que capa se va posicionar cada elemnto del grafico
    for i in range(num_cargas):
        plt.text(cargas[i, 0], cargas[i, 1], f'${cargas[i, 2]:.2e}$',
                 fontsize=10, color='black', ha='center', va='center', zorder=11)
    
    # Calcular la intensidad del campo eléctrico,se le aplico a este valor
    # una transformacion con log con la intencion de marcar mas la diferencia 
    # en los colores de las lineas de flujo en funcion de la intencidad del campo
    # esta aplicacion no es original, la encontre en internet aun que tenia otro uso.
    IC = 2 * np.log(np.hypot(Ex, Ey))
    # Determinar el color de las cargas en función de su valor (+, -)
    color_carga = ['blue' if q < 0 else 'red' for q in cargas[:, 2]]
    
    # Configuración para el uso de fuentes de latex en la grafica
    plt.rcParams.update({'font.size': 14, 'text.usetex': True})
    # Graficar líneas del campo eléctrico
    strm = plt.streamplot(X, Y, Ex, Ey, density=3, linewidth=1, arrowsize=1.5, arrowstyle='->', cmap='CMRmap', color=IC)
    # Graficar las cargas como puntos
    plt.scatter(cargas[:, 0], cargas[:, 1], c=color_carga, marker='o', zorder=10)
    
    # Configuración  de la gráfica
    plt.title(r'Campo eléctrico')
    plt.xlabel(r'Posición x')
    plt.ylabel(r'Posición y')
    # Como consecuencia de la transformacion logaritmica la barra 
    # de color no contiene informacion correcta, no la he retirado 
    # con la esperanza de tener una idea que solucione este error.
    plt.colorbar(strm.lines)
    # Guardar un PDF del grafico generado
    plt.savefig('Cfile5.pdf', format='pdf', dpi=1000, bbox_inches='tight', pad_inches=0.1)

# Pedir al usuario el margen para visualizar las cargas en el gráfico
margen = float(input('Ingrese el margen para visualizar las cargas: '))

# Determinar los límites del gráfico en función de las posiciones de las cargas y el margen
xmax = np.amax(cargas[:, 0]) + margen
xmin = np.amin(cargas[:, 0]) - margen
ymax = np.amax(cargas[:, 1]) + margen
ymin = np.amin(cargas[:, 1]) - margen

# Graficar el campo eléctrico con las cargas
grafico_campo(xmin, xmax, ymin, ymax)